# Développement d’un module de classification d’images IHC par IA

### By Maëlle Broustal, Rafik Mankour & Ian Pellet

## Import

In [1]:
import numpy as np
import pandas as pd
import skimage
import skimage.draw
import matplotlib.pyplot as plt
from xml.dom import minidom as minidom
import PIL 

In [1]:
from tensorflow import keras as k

In [3]:
#k.__version__

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Openslide

In [ ]:
!apt update && apt install -y openslide-tools
!pip install openslide-python

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [7]:
import openslide

ModuleNotFoundError: ignored

## Chargement des images et annotations

In [ ]:
img_test = openslide.OpenSlide('./data/CRC1.ndpi') # open whole-slide image

NameError: ignored

In [ ]:
#Rafik
img_test = openslide.OpenSlide('/content/drive/MyDrive/data/CRC1.ndpi')

In [ ]:
img_test.level_dimensions # dimensions for each levels of the slide

((64512, 53760),
 (32256, 26880),
 (16128, 13440),
 (8064, 6720),
 (4032, 3360),
 (2016, 1680),
 (1008, 840),
 (504, 420),
 (252, 210),
 (126, 105))

In [ ]:
img_test.level_downsamples # downsample rates of each level

(1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0, 512.0)

In [ ]:
img_test.get_thumbnail((1200, 1200)) # RGB image of the slide of size (1200,1200)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# for each level, print level and size and open a region of the slide
max_size = (1800, 1000)
for lvl in range(img_test.level_count):
    lvl_size = img_test.level_dimensions[lvl]
    print(lvl, lvl_size)
    disp_size = max_size if lvl_size > max_size else lvl_size

    img_test.read_region((0,0), lvl, disp_size).show()

0 (64512, 53760)
1 (32256, 26880)
2 (16128, 13440)
3 (8064, 6720)
4 (4032, 3360)
5 (2016, 1680)
6 (1008, 840)
7 (504, 420)
8 (252, 210)
9 (126, 105)


In [ ]:
# we can extract a region of a level of our slide 
PIL_img = img_test.read_region((8000,4000), 5, (1200, 1200))
type(PIL_img)

PIL.Image.Image

In [ ]:
np_img = np.array(PIL_img) # convert PIL Image to a Numpy array
np_img.shape # RGBA

(1200, 1200, 4)

In [ ]:
lvl = 5 # levels 0 to 2 are to big 
full_img = img_test.read_region((0,0), lvl, img_test.level_dimensions[lvl])
full_np = np.array(full_img)
full_np.shape

(1680, 2016, 4)

In [ ]:
img_test.close() # close image

### Annotations

In [ ]:
#pour Rafik
annot = minidom.parse("/content/drive/MyDrive/data/annot.annotations") # open XML file

In [ ]:
#pour Mallou
annot = minidom.parse("/content/drive/MyDrive/Projet 4BIM/data/annot.annotations") # open XML file

In [ ]:
regions = annot.getElementsByTagName("Region") # list of regions in the annotation file
print(len(regions))

26


In [ ]:
V = [] # list of vertices in each region
V_coord = [] # list of coord of each vertex in each region
for r in regions :
  if r.getAttribute("NegativeROA")=='0': 
    vertices = r.getElementsByTagName("V")
    V.append(vertices)
    
    v_coord = []
    for v in vertices :
        coord = {"X": v.getAttribute('X'), "Y": v.getAttribute('Y')}
        v_coord.append(coord)
    V_coord.append(v_coord)

#### 0/1 array representation of regions of interest

In [ ]:
img = np.ones((8064, 6720)) # creates an array of ones of the same shape of our image
ds_rate = 8 # downsample rate

polygons = [] # list of polygons, each object is an array containing the coord of the px in a region of V_coord

for region in V_coord:
  x = [] # list of x coord in the region
  y = [] # list of y coord in the region
  
  for v in region:
    # as we dont take the full resolution slide of the image, we need to divide the coord of each vertice by the downsampling rate
    temp_x = int(v['X'])/ds_rate 
    temp_y = int(v['Y'])/ds_rate
    
    x.append(int(temp_x))
    y.append(int(temp_y))

  # sikimage computes the coord of each px in the region from the vertices defining the perimeter of the region
  polygons.append(skimage.draw.polygon(x,y)) 

In [ ]:
# modify img so that each px in a region of V_coord is set to 0
for p in range(len(polygons)):
  poly = np.transpose(polygons[p])

  for i in range(len(poly)):
    img[(poly[i][0], poly[i][1])] = 0

In [ ]:
img

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

## Mise en place du réseau de neurones

Adapter ce code issu des liens dans le sprint backlog avec des données :
Problème à régler -> faire correspondre la taille et la dimension de l'image ) celle de la matrice d'annotation

In [ ]:
all_images = []
all_images.append(full_np)

all_annotations = []
all_annotations.append(img)

x_train = np.array(all_images)
y_train = np.array(all_annotations)
y_train = k.utils.to_categorical(y_train, 2)

print(np.shape(x_train))
np.shape(y_train)

(1, 1680, 2016, 4)


(1, 8064, 6720, 2)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs

model.fit(x_train, y_train, batch_size=1, epochs=10, verbose=1)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers

model.fit(x_train, y_train, batch_size=1, epochs=10, verbose=1)

# FOURNIR X_TRAIN AVEC LES IMAGES et Y_TRAIN AVEC LES ANNOTATIONS A EMPLOYER

for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers

Epoch 1/10


ValueError: ignored

## Test du réseau de neurones